In [ ]:
# Importando os módulos que serão utilizados

import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import time
import dill

# Configurando o acesso do tensorflow aos processadores do computador
# no caso, está sendo selecionada a primeira placa de vídeo listada

gpus = tf.config.experimental.list_physical_devices('GPU') # Listando as placas de vídeo
tf.config.experimental.set_memory_growth(gpus[0], True)    # Selecionando a primeira GPU e configurando

# Importando os módulos contendo as funções criadas no projeto

from aux_func_V3 import *
import API_Numpy
import API_TensorFlow

In [ ]:
Sim, Sim_step, DerivadaEspacial, Get_weights=create_simulation(API_Numpy,burgers_equation,WENO_Z)

In [ ]:
Δx=0.01

In [ ]:
WENO_Z_ref=lambda u0, Δx:Sim(u0,Δx*0.5, Δx, 0.5, FronteiraFixa)

In [ ]:
np.random.seed(seed=8413651)
n=int(200/2)
k=11
x_rep=20
pesos=np.random.uniform(size=[n,k],low=-1,high=1)
ordem=np.floor(np.random.uniform(size=[n,1],low=1,high=k))+np.asarray([range(k)])
pesos=(1-np.sum(tf.one_hot(ordem,k),axis=1))*pesos

operador_derivada=np.eye(k,k=-1)@np.diag(range(1,k+1))

In [ ]:
x=np.expand_dims(np.arange(-1,1,Δx),axis=0)**np.expand_dims(np.arange(k),axis=1)
Δx_ref=Δx/2
x_ref=np.expand_dims(np.arange(-1,1,Δx/2),axis=0)**np.expand_dims(np.arange(k),axis=1)

In [ ]:
k1 = np.random.uniform(0, 10, [n,1]).astype('int32')   # Amostrando uma frequência aleatória para a função seno
k2 = np.random.uniform(0, 10, [n,1]).astype('int32')   # Amostrando uma frequência aleatória para a função seno
a  = np.random.uniform(0, 1, [n,1]) # Amostrando um peso aleatória para ponderar as funções seno
b  = np.random.uniform(0, 2, [n,1]) # Amostrando um modificador de amplitude aleatório
u1 =     a * tf.math.sin(k1*pi*x[1]) # Gerando pontos de acordo com a primeira função seno
u2 = (1-a) * tf.math.sin(k2*pi*x[1]) # Gerando pontos de acordo com a segunda função seno

y_sin=(b*(u1+u2))

u1 =     a * tf.math.sin(k1*pi*x_ref[1]) # Gerando pontos de acordo com a primeira função seno
u2 = (1-a) * tf.math.sin(k2*pi*x_ref[1]) # Gerando pontos de acordo com a segunda função seno

dy_sin=b*(u1+u2)

y=np.matmul(pesos,x)
y=np.concatenate([y,y_sin],axis=0)

#dy_sin=np.stack(dy_sin,axis=0)
#dy=np.matmul(np.matmul(pesos,operador_derivada),x+Δx/2)
dy=np.matmul(x_ref.T,pesos.T).T
dy=np.concatenate([dy,dy_sin],axis=0)

y_max=np.max(np.abs(y))
y =y/y_max
dy=dy/y_max

n=2*n

In [ ]:
for l in range(3):
    polis=np.floor(np.random.uniform(size=[n,2],low=0,high=y.shape[0]))
    y0=y[polis[:,0].astype('int32')]
    y1=y[polis[:,1].astype('int32')]
    dy0=dy[polis[:,0].astype('int32')]
    dy1=dy[polis[:,1].astype('int32')]
    position=np.floor(np.random.uniform(size=[n],low=0,high=y.shape[1])).astype('int32')
    cuts=np.random.choice([0,1],size=[n],replace=True,p=[0.5,0.5])
    
    for i,j,k in zip(range(n),position,cuts):
        y0[i,j:]=0
        y1[i,:j]=0
        dy0[i,(2*j+k):]=0
        dy1[i,:(2*j+k)]=0
    y=y0+y1
    dy=dy0+dy1

In [ ]:
y_list=[]
dy_list=[]
for i in range(500):
    y=dy[...]
    dy=WENO_Z_ref(y[...],Δx_ref)
    
    y_list.append(y)
    dy_list.append(dy)
    
dy=np.concatenate(dy_list,axis=0)[:,np.arange(x[1].shape[0])*2]
y =np.concatenate( y_list,axis=0)[:,np.arange(x[1].shape[0])*2]

In [ ]:
index=np.floor(np.random.uniform(size=[5],low=0,high=n*100)).astype('int32')

plt.figure(figsize=(6.4*5,6.4*2))
for i,j in zip(range(5),index):
    plt.subplot(2, 5, i+1)
    plt.plot(y[j])
    plt.subplot(2, 5, i+6)
    plt.plot(dy[j])

In [ ]:
with open('datasets\\Dados temporais.bkp','wb') as file:
    dill.dump([y,dy],file)